In [7]:
#This part of code, the goal is to go from mapping and county level weather data to to subgroup data
# Define paths
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
full_future_data_path = '/Volumes/T7/weather2/rcp85hotter/'
full_historical_data_path = '/Volumes/T7/weather2/historic/'

#this just smooth out operation
cutoff_year=2020 #The year prediction dataset start
start_year=2015 #The year historical 
end_year=2100

#Debug mode(only output once)
debug=False
#create population level data
#import require file
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os
from datetime import datetime
from glob import glob

In [3]:
# mapping
mapping_path = data_and_output_dir + 'Input/county_map.csv'
mapping_df = pd.read_csv(mapping_path)
mapping_df 

,county,state,cnty_fips,reeds_region,reeds_ba,reeds_raz,lbnl_region,nrel_region
0,Chelan,Washington,53007,s4,p1,WECC Pacific Northwest,West,Pacific
1,Kittitas,Washington,53037,s4,p1,WECC Pacific Northwest,West,Pacific
2,Island,Washington,53029,s3,p1,WECC Pacific Northwest,West,Pacific
3,King,Washington,53033,s3,p1,WECC Pacific Northwest,West,Pacific
4,Kitsap,Washington,53035,s3,p1,WECC Pacific Northwest,West,Pacific
...,...,...,...,...,...,...,...,...
3113,Aroostook,Maine,23003,s355,p134,NPCC New England,Northeast,Northeast
3114,Franklin,Maine,23007,s355,p134,NPCC New England,Northeast,Northeast
3115,Oxford,Maine,23017,s355,p134,NPCC New England,Northeast,Northeast
3116,Piscataquis,Maine,23021,s355,p134,NPCC New England,Northeast,Northeast


In [8]:



#Path, this will update itself
mapping_path = data_and_output_dir + 'Input/county_map.csv'
future_population_data_path = data_and_output_dir + 'input/Electric_Retail_Service_Territories/ssp3_county_population.csv'
historical_population_data_path = data_and_output_dir + 'input/county_populations_2000_to_2020.csv'
output_path = data_and_output_dir + 'output/'

# Load mapping and population data
mapping_df = pd.read_csv(mapping_path)
future_population_df = pd.read_csv(future_population_data_path)
historical_population_df = pd.read_csv(historical_population_data_path)

#crate looping for interp
years = np.arange(cutoff_year, end_year + 1, 10)
interp_years = np.arange(cutoff_year, end_year + 1)

# Ensure FIPS codes are five zero-padded strings
historical_population_df['FIPS'] = historical_population_df['county_FIPS'].astype(str).str.zfill(5)
future_population_df['FIPS'] = future_population_df['FIPS'].astype(str).str.zfill(5)
mapping_df['cnty_fips'] = mapping_df['cnty_fips'].astype(str).str.zfill(5)

# Rename 'pop_{year}' columns to just '{year}'
for year in range(start_year, cutoff_year+1):
    if f'pop_{year}' in historical_population_df.columns:
        historical_population_df.rename(columns={f'pop_{year}': str(year)}, inplace=True)



# Assuming 'FIPS' is the column to join on and 'Year' is the column indicating the year in future_population_df
for index, row in future_population_df.iterrows():
    pop_values = [row[str(year)] for year in years if str(year) in row]
    interpolator = interp1d(years, pop_values, kind='linear', fill_value="extrapolate")
    interpolated_values = interpolator(interp_years)
    for year in interp_years:
        future_population_df.at[index, str(year)] = interpolated_values[year - cutoff_year]


# Select relevant columns for historical data
historical_population_df = historical_population_df[['FIPS'] + [str(year) for year in range(start_year, cutoff_year)]]

# Select relevant columns for future data
future_population_df = future_population_df[['FIPS'] + [str(year) for year in interp_years]]

# Concatenate historical and future dataframes
combined_population_df = pd.merge(historical_population_df, future_population_df, on='FIPS', how='outer')
# Merge combined population data with mapping data
# Assuming 'GEOID' in mapping_df corresponds to 'FIPS' in population data
combined_df = pd.merge(mapping_df, combined_population_df, left_on='cnty_fips', right_on='FIPS', how='inner')
combined_df['FIPS']=combined_df['cnty_fips']

# Identify FIPS codes in mapping_df that did not successfully join
missing_in_combined = set(mapping_df['cnty_fips']) - set(combined_df['cnty_fips'])
print(missing_in_combined)
for column in combined_df.columns:
    print(column)

{'09140', '09170', '09120', '09150', '09110', '09130', '09160', '09180', '09190'}
county
state
cnty_fips
reeds_region
reeds_ba
reeds_raz
lbnl_region
nrel_region
FIPS
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100


In [9]:
# Extract unique 'reeds_ba' values from combined_df
unique_reeds_ba = set(combined_df['reeds_ba'].unique())

# Generate a set of expected 'reeds_ba' values ('p1' to 'p134')
expected_reeds_ba = set([f'p{i}' for i in range(1, 135)])

# Find missing and extra values
missing_reeds_ba = expected_reeds_ba - unique_reeds_ba
extra_reeds_ba = unique_reeds_ba - expected_reeds_ba

# Print results
if not missing_reeds_ba:
    print("All expected 'reeds_ba' values from 'p1' to 'p134' are present in combined_df.")
else:
    print(f"Missing 'reeds_ba' values in combined_df: {missing_reeds_ba}")

if extra_reeds_ba:
    print(f"Extra 'reeds_ba' values in combined_df not in the range 'p1' to 'p134': {extra_reeds_ba}")


All expected 'reeds_ba' values from 'p1' to 'p134' are present in combined_df.


In [14]:
# main

historical_weather_path = os.path.join(output_path, 'historical_weather')
future_weather_path = os.path.join(output_path, 'future_weather')
os.makedirs(historical_weather_path, exist_ok=True)
os.makedirs(future_weather_path, exist_ok=True)
# Function to load checkpoint
def load_checkpoint(year):
    checkpoint_file = os.path.join(output_path, 'checkpoint.txt')
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as file:
            completed_years = file.read().splitlines()
        return str(year) in completed_years
    return False

# Function to save checkpoint
def save_checkpoint(year):
    checkpoint_file = os.path.join(output_path, 'checkpoint.txt')
    with open(checkpoint_file, 'a') as file:
        file.write(f"{year}\n")
yearloopbreaker=False
fileloopbreaker=False
rbloopbreaker=False
for year in range(start_year, end_year + 1):

    if yearloopbreaker:break
    if load_checkpoint(year):
        print(f"Year {year} already processed. Skipping.")
        continue


    print(f'Process start for year: {year}')
    if year < cutoff_year:
        folder_to_read = os.path.join(full_historical_data_path, str(year))
        folder_to_store = historical_weather_path
    else:
        folder_to_read = os.path.join(full_future_data_path, str(year))
        folder_to_store = future_weather_path
    
    # Initialize a dictionary to hold DataFrames for each rb
    rb_dfs = {}
    # Sort files before processing
    files = sorted(glob(os.path.join(folder_to_read, '*.csv')))
    total_files = len(files)
    processed_files = 0

    # Iterate through each file in the year's folder
    for file_path in files:
        processed_files += 1
        #print(f"Processing file {processed_files}/{total_files} ({(processed_files/total_files)*100:.2f}%)")

        if fileloopbreaker:break
        
        meteorology_df = pd.read_csv(file_path)
        if meteorology_df.empty:
            print(f"Skipped empty file: {file_path}")
            continue
        #print(meteorology_df.columns)
        meteorology_df['FIPS'] = meteorology_df['FIPS'].astype(str).str.zfill(5)
        meteorology_df['WSPD'] = np.sqrt(meteorology_df['U10']**2 + meteorology_df['V10']**2)
        
        # Extract date and time from file name
        _, month, day, hour = map(int, os.path.basename(file_path).split('_')[:4])
        utc_time = datetime(year, month, day, hour).strftime('%Y-%m-%d %H:%M:%S')
        meteorology_df['Time_UTC'] = utc_time
        
        # Merge with background data
        merged_df = pd.merge(combined_df, meteorology_df, on='FIPS', how='inner')
        
        # Calculate weighted averages and accumulate in rb_dfs
        merged_df['Population'] = merged_df[str(year)]
        for rb, group in merged_df.groupby('reeds_ba'):
            if rbloopbreaker:break
            
            if rb not in rb_dfs:
                rb_dfs[rb] = pd.DataFrame()
            weighted = group.copy()
            
            for col in ['T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']:
                weighted[col] = weighted[col] * weighted['Population']
            weighted_sum = weighted.groupby('reeds_ba').sum()
            for col in ['T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']:
                weighted_sum[col] = weighted_sum[col] / weighted_sum['Population']
            weighted_sum['Time_UTC'] = utc_time
            weighted_sum=weighted_sum[['Time_UTC','T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']]
            rb_dfs[rb] = pd.concat([rb_dfs[rb], weighted_sum.reset_index()], ignore_index=True)
        if debug:
            fileloopbreaker=True
            yearloopbreaker=True
    save_checkpoint(year)

    
    # Save each RB's accumulated data to its file
    for rb, df in rb_dfs.items():
        output_file = os.path.join(folder_to_store, f"{rb}_WRF_Hourly_Mean_Meteorology_{year}.csv")
        df.sort_values(by='Time_UTC', inplace=True)
        df[['Time_UTC','T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']].to_csv(output_file, index=False)
        print(f"Processed and saved: {output_file} for RB: {rb}")

Process start for year: 2015
